# Imports

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from sklearn.metrics import mean_squared_error

# Load Data

In [2]:
from helper_functions import subject_df

/Users/djw/Documents/pCloud_synced/Academics/Projects/2020_thesis/thesis_experiments/3_experiments/3_3_experience_sampling/3_3_3_data_analysis/02_analysis/helper_functions.py:23: DtypeWarning: Columns (45) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../02_analysis/df_good.csv')


In [3]:
import warnings

warnings.filterwarnings('ignore')

# Function to load all participants
df_good = pd.read_csv('../02_analysis/df_good.csv')

df = subject_df(df=df_good, sub_num=0)

### Target column

In [4]:
# Define a function to calculate the 3-day moving average with forward-looking window
def forward_moving_average(series, window=3):
    return series.rolling(window=window, min_periods=1).mean().shift(-window)

# Apply the moving average calculation for each PID
df_good['target'] = df_good.groupby('PID')['sr_gap_heuristic'].transform(lambda x: forward_moving_average(x, window=3))

In [5]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import TimeSeriesSplit, cross_val_score
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from scipy.stats import binom_test

# Initialize an empty DataFrame to store the results
results_df = pd.DataFrame(columns=['subject', 'logistic_reg_cv', 'random_forest_cv', 'xgboost_cv', 'ensemble_cv', 'logistic_reg_test', 'random_forest_test', 'xgboost_test', 'ensemble_test', 'p_val_xgb_cv', 'ffill', 'xgb_predictions'])

# How many features to use
num_features = 15

for i in range(len(np.unique(df_good.PID))):
    print("Subject", i)
    
    try:
        df_split = subject_df(df=df_good, sub_num=i)

        # Use pd.qcut to create categories
        df_split['y_cat'], bins = pd.qcut(df_split['target'], q=2, labels=['Low', 'High'], retbins=True)

        # Make boolean
        df_split['y_cat_high'] = df_split['y_cat'].cat.codes
        df_split['y_cat_high'] = df_split['y_cat_high'].replace(-1, np.nan)

        # Target column is next day's gap
        df_split['y_cat_highNextDay'] = df_split['y_cat_high'].shift(-1)

        # Drop non-boolean cat
        df_split.drop(columns='y_cat', inplace=True)

        # Remove trailing rows since no y1 value
        df_split = df_split.iloc[:-2]

        # Separate features and target variable
        X = df_split.drop(columns=['y_cat_highNextDay'])
        y = df_split['y_cat_highNextDay']

        # Remove initial rows with NaN values for y_cat_high
        while y.isnull().iloc[0]:
            X = X.iloc[1:].reset_index(drop=True)
            y = y.iloc[1:].reset_index(drop=True)
        
        # Remove all "target" columns
        X = X.loc[:, ~X.columns.str.contains('target')]
        
        # Count missing values before imputation
        missing_values_before = y.isnull().sum()

        # Impute missing values in the target variable using forward fill
        y = y.fillna(method='ffill')

        # Count missing values after imputation
        missing_values_after = y.isnull().sum()

        # Calculate the number of imputed values
        imputed_value_count = missing_values_before - missing_values_after

        # Ensure there are no more NaN values in y
        if y.isnull().sum() == 0:
            # Train-test split for time series data
            split_ratio = 0.8
            split_index = int(len(X) * split_ratio)

            X_train, X_test = X[:split_index], X[split_index:]
            y_train, y_test = y[:split_index], y[split_index:]

            # Handle missing values and standardize features
            imputer = SimpleImputer(strategy='mean')
            scaler = StandardScaler()

            # Feature selection
            k_best = SelectKBest(score_func=f_classif, k=num_features)

            # Models
            log_reg = LogisticRegression(max_iter=1000)
            rf = RandomForestClassifier(n_estimators=100)
            xgb_clf = xgb.XGBClassifier(n_estimators=100, use_label_encoder=False, eval_metric='logloss')

            # TimeSeriesSplit
            tscv = TimeSeriesSplit(n_splits=5)

            # Pipelines
            log_reg_pipeline = Pipeline([
                ('imputer', imputer),
                ('scaler', scaler),
                ('k_best', k_best),
                ('classifier', log_reg)
            ])

            rf_pipeline = Pipeline([
                ('imputer', imputer),
                ('scaler', scaler),
                ('k_best', k_best),
                ('classifier', rf)
            ])

            xgb_pipeline = Pipeline([
                ('imputer', imputer),
                ('scaler', scaler),
                ('k_best', k_best),
                ('classifier', xgb_clf)
            ])

            # Define the parameter grids for each model
            param_grid_log_reg = {
                'classifier__C': [0.01, 0.1, 1, 10, 100]
            }

            param_grid_rf = {
                'classifier__n_estimators': [50, 100, 200],
                'classifier__max_depth': [None, 10, 20, 30]
            }

            param_grid_xgb = {
                'classifier__n_estimators': [50, 100, 200],
                'classifier__learning_rate': [0.01, 0.1, 0.2],
                'classifier__max_depth': [3, 6, 9],
                'classifier__reg_alpha': [0, 0.1, 0.5],
                'classifier__reg_lambda': [0, 0.5, 1]
            }

            # Setup GridSearchCV for each model
            grid_log_reg = GridSearchCV(log_reg_pipeline, param_grid_log_reg, cv=tscv, scoring='accuracy')
            grid_rf = GridSearchCV(rf_pipeline, param_grid_rf, cv=tscv, scoring='accuracy')
            grid_xgb = GridSearchCV(xgb_pipeline, param_grid_xgb, cv=tscv, scoring='accuracy')

            # Fit the models with cross-validation
            grid_log_reg.fit(X_train, y_train)
            grid_rf.fit(X_train, y_train)
            grid_xgb.fit(X_train, y_train)

            # Print best parameters for the XGBoost model
            print("Best parameters for XGBoost:", grid_xgb.best_params_)

            # Best estimators from cross-validation
            best_log_reg = grid_log_reg.best_estimator_
            best_rf = grid_rf.best_estimator_
            best_xgb = grid_xgb.best_estimator_

            # Ensemble with VotingClassifier
            voting_clf = VotingClassifier(estimators=[
                ('log_reg', best_log_reg),
                ('rf', best_rf),
                ('xgb', best_xgb)
            ], voting='soft')

            # Fit the ensemble model
            voting_clf.fit(X_train, y_train)

            # Cross-validation scores for ensemble model
            ensemble_cv_scores = cross_val_score(voting_clf, X_train, y_train, cv=tscv, scoring='accuracy')

            # Predictions and accuracy on the held-out test set
            y_pred_log_reg = best_log_reg.predict(X_test)
            y_pred_rf = best_rf.predict(X_test)
            y_pred_xgb = best_xgb.predict(X_test)
            y_pred_ensemble = voting_clf.predict(X_test)

            log_reg_accuracy_test = accuracy_score(y_test, y_pred_log_reg)
            rf_accuracy_test = accuracy_score(y_test, y_pred_rf)
            xgb_accuracy_test = accuracy_score(y_test, y_pred_xgb)
            ensemble_accuracy_test = accuracy_score(y_test, y_pred_ensemble)

            # Cross-validation accuracies
            log_reg_accuracy_cv = grid_log_reg.best_score_
            rf_accuracy_cv = grid_rf.best_score_
            xgb_accuracy_cv = grid_xgb.best_score_
            ensemble_accuracy_cv = ensemble_cv_scores.mean()

            # Store predictions and actual values in a dictionary for XGBoost
            prediction_dict = [{'predicted_value': pred, 'actual_value': actual} for pred, actual in zip(y_pred_xgb, y_test)]

            # Number of test samples
            n = len(y_test)
            # Number of correct predictions (assuming ensemble model predictions)
            k = int(xgb_accuracy_cv * n)
            p_chance = 0.5  # Chance level accuracy

            # Perform the binomial test
            p_value = binom_test(k, n, p_chance, alternative='greater')

            # Save results to the DataFrame using pd.concat
            new_row = pd.DataFrame({
                'subject': [i],
                'logistic_reg_cv': [log_reg_accuracy_cv],
                'random_forest_cv': [rf_accuracy_cv],
                'xgboost_cv': [xgb_accuracy_cv],
                'ensemble_cv': [ensemble_accuracy_cv],
                'logistic_reg_test': [log_reg_accuracy_test],
                'random_forest_test': [rf_accuracy_test],
                'xgboost_test': [xgb_accuracy_test],
                'ensemble_test': [ensemble_accuracy_test],
                'p_val_xgb_cv': [p_value],
                'ffill': [imputed_value_count],
                'xgb_predictions': [prediction_dict]
            })
            results_df = pd.concat([results_df, new_row], ignore_index=True)
            
        else:
            print(f"Skipping subject {i} because the target variable y still contains NaN values after imputation.")
            continue
    except KeyError as err:
        print(f'Error with subject {i}')


Subject 0
